In [ ]:
import os
import time

import numpy as np

import matplotlib
import matplotlib.animation 
import matplotlib.pyplot as plt

#import skimage
#import skimage.io as sio

from functional import ft_convolve

import IPython

In [ ]:
"""
functions for building CA physics
"""

def make_gaussian(a, m, s):
    eps = 1e-9
    def gaussian(x):
        
        return a * np.exp(-((x-m)/(eps+s))**2 / 2)

    return gaussian
    
def make_mixed_gaussian(amplitudes, means, std_devs):
    
    def gaussian_mixture(x):
        results = 0.0 * x
        eps = 1e-9 # prevent division by 0
        
        for a, m, s in zip(amplitudes, means, std_devs):
            my_gaussian = make_gaussian(a, m, s)
            results += my_gaussian(x)
        
        return results

    return gaussian_mixture

def make_kernel_field(kernel_radius, dim=126):

    #dim = kernel_radius * 2 + 1
    

    x =  np.arange(-dim / 2, dim / 2 + 1, 1)
    xx, yy = np.meshgrid(x,x)

    rr = np.sqrt(xx**2 + yy**2) / kernel_radius

    return rr

def make_update_function(mean, standard_deviation):

    my_gaussian = make_gaussian(1.0, mean, standard_deviation)
    
    def lenia_update(x):
        """
        lenia update
        """
        return 2 * my_gaussian(x) - 1

    return lenia_update

def make_update_step(update_function, kernel, dt, clipping_function = lambda x: x, decimals=None):

    if decimals is not None:
        r = lambda x: np.round(x, decimals=decimals)
    else:
        r = lambda x: x
        
    def update_step(grid):

        
        neighborhoods = r(ft_convolve(r(grid), r(kernel)))
        dgrid_dt = r(update_function(neighborhoods))

        new_grid = r(clipping_function(r(grid) + dt * dgrid_dt))

        return new_grid

    return update_step

def make_make_kernel_function(amplitudes, means, standard_deviations):
    
    def make_kernel(kernel_radius, dim=127):
                
        gm = make_mixed_gaussian(amplitudes, means, standard_deviations)
        rr = make_kernel_field(kernel_radius)
        kernel = gm(rr)[None,None,:,:]
        kernel /= kernel.sum()

        return kernel

    return make_kernel

In [ ]:
"""
animation functions
"""

def get_fig(grid):
    
    global subplot_0
    
    fig, ax = plt.subplots(1,1)
    
    subplot_0 = ax.imshow(grid.squeeze(), cmap="magma")
    
    return fig, ax

def update_frame(ii):
    
    global grid
    
    subplot_0.set_array(grid.squeeze())
    
    grid = update_step(grid)


In [ ]:
# common setup

# the neighborhood kernel
amplitudes = [0.5, 1.0, 0.6667]
means = [0.0938, 0.2814, 0.4690]
standard_deviations = [0.0330, 0.0330, 0.0330]
kernel_radius = 31

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
kernel = make_kernel(kernel_radius)

# the growth function
mean_g = 0.26
standard_deviation_g = 0.036

clipping_fn = lambda x: np.clip(x,0,1.0)
my_update = make_update_function(mean_g, standard_deviation_g)

plt.figure()
plt.imshow(kernel.squeeze())
plt.title("${\it H. natans}$ neighborhood kernel")

x = np.arange(0, 1.01, 0.01)
y = my_update(x)
plt.figure()
plt.plot(x, y, lw=4)
plt.title("${\it H. natans}$ growth function")

In [ ]:
def stability_sweep(dts, krs, starting_grid, make_kernel, my_update, max_t, \
                    max_steps=1000, \
                   max_growth=2,\
                   min_growth=0.5):


    results = np.zeros((dts.shape[0], krs.shape[0],3))
    max_growth = 1.5
    min_growth = 0.5
    

    starting_sum = starting_grid.sum()
    
    for ii, dt in enumerate(dts):
        for jj, kr in enumerate(krs):

            kernel = make_kernel(kr)
            update_step = make_update_step(my_update, kernel, dt, clipping_fn)

            accumulated_t = 0.0
            total_steps = 0
            grid = starting_grid * 1.0
            explode = False
            vanish = False
            
            while accumulated_t < max_t and total_steps <= max_steps:
 
                grid = update_step(grid)
                
                g = grid.sum() / starting_sum
                
                accumulated_t += dt
                total_steps += 1
                if g > max_growth:
                    explode = True
                    break
                if g < min_growth:
                    vanish = True
                    break
                    
            if explode == True:
                results[ii,jj,0] = 1-accumulated_t / max_t
            elif vanish == True:
                results[ii,jj,2] = 1-accumulated_t / max_t
            else:
                results[ii,jj,1] = accumulated_t / max_t

    return results

    

# The Platonic Pattern: _Hydrogeminium natans_ pickle

In [ ]:
pattern_filepath = os.path.join("patterns", "hydrogeminium_natans_pickle.npy")

pattern = np.load(pattern_filepath)[None,None,:,:]
plt.figure()
plt.imshow(pattern.squeeze(), cmap="magma")


# The Non-Platonic _H. natans_ wobbler

In [ ]:
#
pattern_filepath = os.path.join("patterns", "hydrogeminium_natans_wobbler.npy")

pattern = np.load(pattern_filepath)[None,None,:,:]
plt.figure()
plt.imshow(pattern.squeeze(), cmap="magma")


In [ ]:
min_dt = 0.0001
max_dt = 1.0
number_dt_steps = 64
min_kr = 1
max_kr = 63
number_kr_steps = 64

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.max() * 100

number_samples = 1
dim = 196
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
t0 = time.time()
results = stability_sweep(dts, krs, grid, make_kernel, my_update, max_t)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.20
max_dt = 0.45
number_dt_steps = 256
min_kr = 11
max_kr = 33
number_kr_steps = 256

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.min() * 100

number_samples = 1
dim = 196
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
t0 = time.time()
results = stability_sweep(dts, krs, grid, make_kernel, my_update, max_t)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.37
max_dt = 0.42
number_dt_steps = 256
min_kr = 23.3
max_kr = 26.8
number_kr_steps = 256

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.min() * 10

number_samples = 1
dim = 196
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
t0 = time.time()
results = stability_sweep(dts, krs, grid, make_kernel, my_update, max_t)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.425
max_dt = 0.435
number_dt_steps = 256
min_kr = 25
max_kr = 25.8
number_kr_steps = 256

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.min() * 100

number_samples = 1
dim = 196
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
t0 = time.time()
results = stability_sweep(dts, krs, grid, make_kernel, my_update, max_t)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.4303
max_dt = 0.4309
number_dt_steps = 256
min_kr = 25.2
max_kr = 25.25
number_kr_steps = 256

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.min() * 100

number_samples = 1
dim = 196
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
t0 = time.time()
results = stability_sweep(dts, krs, grid, make_kernel, my_update, max_t)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
min_dt = 0.43043
max_dt = 0.43044
number_dt_steps = 256
min_kr = 25.2219
max_kr = 25.2221
number_kr_steps = 256

dts = np.arange(min_dt, max_dt, (max_dt-min_dt) / number_dt_steps)
krs = np.arange(min_kr, max_kr, (max_kr-min_kr) / number_kr_steps)
max_t = dts.min() * 100

number_samples = 1
dim = 196
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

make_kernel = make_make_kernel_function(amplitudes, means, standard_deviations)
t0 = time.time()
results = stability_sweep(dts, krs, grid, make_kernel, my_update, max_t)
t1 = time.time()

print(f"elapsed: {t1-t0:.4f}")

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4e}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4e}" for elem in krs], rotation=90)

In [ ]:

fig, ax = plt.subplots(1,1, figsize=(20,20))
ax.imshow(results)
_ = ax.set_yticks(np.arange(0,dts.shape[0]))
_ = ax.set_yticklabels([f"{elem:.4f}" for elem in dts])
_ = ax.set_xticks(np.arange(0,krs.shape[0]))
_ = ax.set_xticklabels([f"{elem:.4f}" for elem in krs], rotation=90)

In [ ]:
number_samples = 1
dim = 256
grid = np.zeros((number_samples,1,dim,dim))
grid[:,:,:pattern.shape[-2], :pattern.shape[-1]] = pattern

dts = 0.38


clipping_fn = lambda x: np.clip(x,0,1.0)
update_step = make_update_step(my_update, kernel, dts, clipping_fn, decimals=32)

num_frames = 100

fig, ax = get_fig(grid[0])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_frame, frames=num_frames, interval=100).to_jshtml())